# 事前準備


## matplotlib 日本語化


In [1]:
!pip install japanize-matplotlib


## ライブラリのインポート


In [2]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression as LR
import japanize_matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re
# データを表示する時に列を省略しないためのおまじない
pd.options.display.max_columns = None

%matplotlib inline

# #日本語化matplotlibのインポート & 日本語のフォントを指定
plt.rcParams['font.family'] = 'IPAexGothic'


# データの読み込み


In [3]:
train = pd.read_csv("./input/train_all.csv")
test = pd.read_csv("./input/test_all.csv")

# 投稿用のサンプルファイルの読み込み
sample = pd.read_csv("./input/sample_submit.csv", header=None)


# 前処理


訓練データとテストデータをけつごうしたいが、テストデータに y が無いから先に取り出しておく


In [4]:
y_train = train["y"]
train = train.drop("y", axis=1)


## 結合


In [5]:
# 結合前にtrainというフラグを作り、trainデータは1、testデータは0とする
train["train"] = 1
test["train"] = 0

# 結合
datas = pd.concat([train, test], sort=False)


## 不要なカラムの削除


In [6]:
drop_list = [
    "tv",
    "referee",
    "time",
    "home",
    "away",
    "home_01", "home_02", "home_03", "home_04", "home_05", "home_06", "home_07", "home_08", "home_09", "home_10", "home_11",
    "away_01", "away_02", "away_03", "away_04", "away_05", "away_06", "away_07", "away_08", "away_09", "away_10", "away_11",
    "address",
    "humidity",
]
datas = datas.drop(drop_list, axis=1)

datas.head(5)

,id,year,stage,match,gameday,stadium,home_score,away_score,weather,temperature,home_team,away_team,capa,train
0,13994,2012,Ｊ１,第１節第１日,03/10(土),ユアテックスタジアム仙台,1,0,雨,3.8,ベガルタ仙台,鹿島アントラーズ,19694,1
1,13995,2012,Ｊ１,第１節第１日,03/10(土),豊田スタジアム,1,0,屋内,12.4,名古屋グランパス,清水エスパルス,40000,1
2,13996,2012,Ｊ１,第１節第１日,03/10(土),万博記念競技場,2,3,晴一時雨,11.3,ガンバ大阪,ヴィッセル神戸,21000,1
3,13997,2012,Ｊ１,第１節第１日,03/10(土),エディオンスタジアム広島,1,0,曇一時雨のち晴,11.4,サンフレッチェ広島,浦和レッズ,50000,1
4,13998,2012,Ｊ１,第１節第１日,03/10(土),札幌ドーム,0,0,屋内,22.5,コンサドーレ札幌,ジュビロ磐田,39232,1


## 外れ値の除外

In [7]:
train = train[train["id"] != 15127]	# yの値が大きく外れたデータ
train = train[train["id"] != 15699]	# yの値が0のデータ
train = train[train["id"] != 14071]	# 木の中でyの値が外れたデータ
train = train[train["id"] != 14911]	# 金の中でyの値が外れたデータ

## 日付関係


In [8]:
datas["MONTH"] = datas["gameday"].apply(lambda x: x[0:2])
datas["DAY"] = datas["gameday"].apply(lambda x: x[3:5])
datas["WEEK"] = datas["gameday"].apply(lambda x: x[x.find("(")+1:x.find("(")+2])
datas = pd.get_dummies(datas, columns=["WEEK"], drop_first=True)
datas = datas.drop("gameday", axis=1)


## 試合？関係


In [9]:
datas["stage"] = datas["stage"].apply(lambda x: x[1])
datas["match_num"] = datas["match"].apply(
    lambda x: int(re.findall(r'\d+', x)[0]))
datas["match_day"] = datas["match"].apply(
    lambda x: int(re.findall(r'\d+', x)[1]))
datas = datas.drop("match", axis=1)


## チーム関係


In [10]:
datas = pd.get_dummies(datas, columns=["home_team"], drop_first=True)
datas = pd.get_dummies(datas, columns=["away_team"], drop_first=True)


## スタジアム関係


In [11]:
datas = pd.get_dummies(datas, columns=["stadium"], drop_first=True)


## 説明変数および目的変数の抽出


- 学習データ（train）データからは説明変数と目的変数を、評価データ（test）からは説明変数のみを取り出します
- サンプルコードでは次のように変数の名前を付けます<br>
  - 学習データの説明変数： trainX
  - 学習データの目的変数： y
  - 評価データの説明変数： testX


In [12]:
datas.columns


Index(['id', 'year', 'stage', 'home_score', 'away_score', 'weather',
       'temperature', 'capa', 'train', 'MONTH',
       ...
       'stadium_長崎市総合運動公園かきどまり陸上競技場', 'stadium_長崎県立総合運動公園陸上競技場',
       'stadium_香川県立丸亀競技場', 'stadium_駒沢オリンピック公園総合運動場陸上競技場',
       'stadium_鳴門・大塚スポーツパーク　ポカリスエットスタジアム', 'stadium_鹿児島県立鴨池陸上競技場 ',
       'stadium_ＩＡＩスタジアム日本平', 'stadium_ＮＡＣＫ５スタジアム大宮', 'stadium_ＮＤソフトスタジアム山形',
       'stadium_Ｓｈｏｎａｎ　ＢＭＷスタジアム平塚'],
      dtype='object', length=161)

In [13]:
datas.head()


,id,year,stage,home_score,away_score,weather,temperature,capa,train,MONTH,DAY,WEEK_日,WEEK_月,WEEK_木,WEEK_水,WEEK_火,WEEK_金,match_num,match_day,home_team_アルビレックス新潟,home_team_カターレ富山,home_team_カマタマーレ讃岐,home_team_ガイナーレ鳥取,home_team_ガンバ大阪,home_team_ギラヴァンツ北九州,home_team_コンサドーレ札幌,home_team_サガン鳥栖,home_team_サンフレッチェ広島,home_team_ザスパクサツ群馬,home_team_ザスパ草津,home_team_ジェフユナイテッド千葉,home_team_ジュビロ磐田,home_team_セレッソ大阪,home_team_ファジアーノ岡山,home_team_ベガルタ仙台,home_team_モンテディオ山形,home_team_ロアッソ熊本,home_team_ヴァンフォーレ甲府,home_team_ヴィッセル神戸,home_team_京都サンガF.C.,home_team_名古屋グランパス,home_team_大分トリニータ,home_team_大宮アルディージャ,home_team_川崎フロンターレ,home_team_徳島ヴォルティス,home_team_愛媛ＦＣ,home_team_東京ヴェルディ,home_team_松本山雅ＦＣ,home_team_柏レイソル,home_team_栃木ＳＣ,home_team_横浜Ｆ・マリノス,home_team_横浜ＦＣ,home_team_水戸ホーリーホック,home_team_浦和レッズ,home_team_清水エスパルス,home_team_湘南ベルマーレ,home_team_鹿島アントラーズ,home_team_ＦＣ岐阜,home_team_ＦＣ東京,home_team_ＦＣ町田ゼルビア,home_team_Ｖ・ファーレン長崎,away_team_アルビレックス新潟,away_team_カターレ富山,away_team_カマタマーレ讃岐,away_team_ガイナーレ鳥取,away_team_ガンバ大阪,away_team_ギラヴァンツ北九州,away_team_コンサドーレ札幌,away_team_サガン鳥栖,away_team_サンフレッチェ広島,away_team_ザスパクサツ群馬,away_team_ザスパ草津,away_team_ジェフユナイテッド千葉,away_team_ジュビロ磐田,away_team_セレッソ大阪,away_team_ファジアーノ岡山,away_team_ベガルタ仙台,away_team_モンテディオ山形,away_team_ロアッソ熊本,away_team_ヴァンフォーレ甲府,away_team_ヴィッセル神戸,away_team_京都サンガF.C.,away_team_名古屋グランパス,away_team_大分トリニータ,away_team_大宮アルディージャ,away_team_川崎フロンターレ,away_team_徳島ヴォルティス,away_team_愛媛ＦＣ,away_team_東京ヴェルディ,away_team_松本山雅ＦＣ,away_team_柏レイソル,away_team_栃木ＳＣ,away_team_横浜Ｆ・マリノス,away_team_横浜ＦＣ,away_team_水戸ホーリーホック,away_team_浦和レッズ,away_team_清水エスパルス,away_team_湘南ベルマーレ,away_team_鹿島アントラーズ,away_team_ＦＣ岐阜,away_team_ＦＣ東京,away_team_ＦＣ町田ゼルビア,away_team_Ｖ・ファーレン長崎,stadium_とりぎんバードスタジアム,stadium_エコパスタジアム,stadium_エディオンスタジアム広島,stadium_キンチョウスタジアム,stadium_ケーズデンキスタジアム水戸,stadium_シティライトスタジアム,stadium_デンカビッグスワンスタジアム,stadium_ニッパツ三ツ沢球技場,stadium_ニンジニアスタジアム,stadium_ノエビアスタジアム神戸,stadium_フクダ電子アリーナ,stadium_ベストアメニティスタジアム,stadium_ヤマハスタジアム（磐田）,stadium_ヤンマースタジアム長居,stadium_ユアテックスタジアム仙台,stadium_レベルファイブスタジアム,stadium_万博記念競技場,stadium_京都市西京極総合運動公園陸上競技場兼球技場,stadium_佐賀県総合運動場陸上競技場,stadium_北九州市立本城陸上競技場,stadium_名古屋市瑞穂陸上競技場,stadium_味の素スタジアム,stadium_味の素フィールド西が丘,stadium_国立競技場,stadium_埼玉スタジアム２００２,stadium_大分銀行ドーム,stadium_大垣市浅中公園総合グラウンド陸上競技場,stadium_宮城スタジアム,stadium_富山県総合運動公園陸上競技場,stadium_山梨中銀スタジアム,stadium_岐阜メモリアルセンター長良川球技メドウ,stadium_岐阜メモリアルセンター長良川競技場,stadium_岡山県津山陸上競技場,stadium_日産スタジアム,stadium_日立柏サッカー場,stadium_札幌ドーム,stadium_札幌厚別公園競技場,stadium_松本平広域公園総合球技場,stadium_栃木県グリーンスタジアム,stadium_正田醤油スタジアム群馬,stadium_熊本市水前寺競技場,stadium_熊谷スポーツ文化公園陸上競技場,stadium_町田市立陸上競技場,stadium_県立カシマサッカースタジアム,stadium_神戸総合運動公園ユニバー記念競技場,stadium_福島県営あづま陸上競技場,stadium_等々力陸上競技場,stadium_豊田スタジアム,stadium_長崎市総合運動公園かきどまり陸上競技場,stadium_長崎県立総合運動公園陸上競技場,stadium_香川県立丸亀競技場,stadium_駒沢オリンピック公園総合運動場陸上競技場,stadium_鳴門・大塚スポーツパーク ポカリスエットスタジアム,stadium_鹿児島県立鴨池陸上競技場,stadium_ＩＡＩスタジアム日本平,stadium_ＮＡＣＫ５スタジアム大宮,stadium_ＮＤソフトスタジアム山形,stadium_Ｓｈｏｎａｎ ＢＭＷスタジアム平塚
0,13994,2012,１,1,0,雨,3.8,19694,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,13995,2012,１,1,0,屋内,12.4,40000,1,03,10,False,False,False,False,False,False,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Tru

In [14]:
# 使わない説明変数を設定
drop_list = [
    "weather",
    "year",
    "MONTH",
    "DAY",
]


In [15]:
datas = datas.drop(drop_list, axis=1)


In [16]:
# datasにおいてtrainフラグが1のものをX_trainとする。その後、trainフラグを削除
X_train = datas[datas["train"] == 1]
X_train = X_train.drop(columns="train")

# datasにおいてtrainフラグが0のものをX_testとする。その後、trainフラグを削除
X_test = datas[datas["train"] == 0]
X_test = X_test.drop(columns="train")


# モデルごと


## LinearRegression


### 学習と予測


In [17]:
LRmodel = LR()
LRmodel.fit(X_train, y_train)
pred_LR = LRmodel.predict(X_test)


### 予測値の保存


In [18]:
sample[1] = pred_LR
sample.to_csv("LR.csv", index=None, header=None)


## LinearRegression


### 学習と予測


In [19]:
LRmodel = LR()
LRmodel.fit(X_train, y_train)
pred_LR = LRmodel.predict(X_test)


### 予測値の保存


In [20]:
sample[1] = pred_LR
sample.to_csv("LR.csv", index=None, header=None)


## Ridge


In [21]:
ridge = Ridge()
ridge.fit(X_train, y_train)
pred_ridge = ridge.predict(X_test)
sample[1] = pred_ridge
sample.to_csv("Ridge.csv", index=None, header=None)

ridge10 = Ridge(alpha=10)
ridge10.fit(X_train, y_train)
pred_ridge10 = ridge10.predict(X_test)
sample[1] = pred_ridge10
sample.to_csv("Ridge10.csv", index=None, header=None)

ridge01 = Ridge(alpha=0.1)
ridge01.fit(X_train, y_train)
pred_ridge01 = ridge01.predict(X_test)
sample[1] = pred_ridge01
sample.to_csv("Ridge01.csv", index=None, header=None)


## SVR


In [22]:
regr = SVR()
regr.fit(X_train, y_train)
pred_regr = regr.predict(X_test)
sample[1] = pred_regr
sample.to_csv("regr.csv", index=None, header=None)
